In [2]:
import pandas as pd
import os

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../../"))

In [58]:
data_path = os.path.join(BASE_DIR, "data/raw/data.csv")

data = pd.read_csv(data_path)

In [59]:
data

,Sentence,Emotion
0,cho tao hỏi lúa nó dính mưa vầy ròi thì làm đư...,Trung lập
1,"giảng_viên dạy tận_tình , cung_cấp đầy_đủ kiến...",Vui vẻ
2,per không bao giờ quên được hihi,Vui vẻ
3,"thầy rất nhiệt_tình , quan_tâm sinh_viên , tr...",Vui vẻ
4,"giảng_dạy dễ hiểu, vui_vẻ.",Vui vẻ
...,...,...
28731,thầy rất nhiệt_tình ạ .,Vui vẻ
28732,"thầy cho slide , textbook , bài_tập để nghiên_...",Vui vẻ
28733,tận_tâm hướng_dẫn sinh_viên .,Vui vẻ
28734,em thích đi học tiết của thầy nhất .,Vui vẻ


In [61]:
data['Emotion'] = data['Emotion'].str.strip().str.replace('"', '')

data['Emotion'] = data['Emotion'].replace({
    ' Vui vẻ': 'Vui vẻ',
    ' Trung lập': 'Trung lập',
    ' Tức giận': 'Tức giận',
    ' Buồn bã': 'Buồn bã',
    ' Sợ hãi': 'Sợ hãi',
    '"Vui vẻ"': 'Vui vẻ',
    '"Tức giận"': 'Tức giận',
    '"Buồn bã"': 'Buồn bã',
    '"Sợ hãi"': 'Sợ hãi'
})

In [62]:
data['Emotion'].value_counts()

Emotion
Vui vẻ       9595
Trung lập    5509
Sợ hãi       5203
Tức giận     4255
Buồn bã      4174
Name: count, dtype: int64

In [63]:
data_path = os.path.join(BASE_DIR, "data/raw/data.csv")

data.to_csv(data_path, index=False)

In [64]:
df_shuffled = data.sample(frac=1, random_state=42).reset_index(drop=True)

data_suffled_path = os.path.join(BASE_DIR, "data/raw/data.csv")

df_shuffled.to_csv(data_suffled_path, index=False)

In [65]:
data = pd.read_csv(data_path)
data['Emotion'].value_counts()

Emotion
Vui vẻ       9595
Trung lập    5509
Sợ hãi       5203
Tức giận     4255
Buồn bã      4174
Name: count, dtype: int64

# Chuẩn hóa nhãn thành sốsố

In [66]:
label_mapping = {
    "Vui vẻ": 0,
    "Tức giận": 1,
    "Buồn bã": 2,
    "Sợ hãi": 3,
    "Trung lập": 4
}

data["Label"] = data["Emotion"].map(label_mapping)
data = data.drop(columns=["Emotion"])
data


,Sentence,Label
0,"bên cạnh đó còn có rèn kỹ_năng viết , kỹ_năng ...",0
1,"nhiệt_tình , dễ hiểu , đúng giờ .",0
2,những gì được học nó nhằm mục_đích gì ?,4
3,xinh quá !,0
4,"giảng_viên rất vui_vẻ , thân_thiện , nhiệt_tìn...",0
...,...,...
28731,chất_lượng môn_học phù_hợp .,0
28732,tao bực mày lắm rồi đấy,1
28733,per anh ơi đừng gym nữa . boxing là đụ rùi :((,2
28734,thầy dạy vui lắm .,0


# Loại bỏ ký tự đặc biệt, số và khoảng trắng thừa 

In [67]:
import re

def clean_text(text):
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"[^\w\s]", "", text)
    return text

data["Sentence"] = data["Sentence"].astype(str).apply(clean_text)
data

,Sentence,Label
0,bên cạnh đó còn có rèn kỹ_năng viết kỹ_năng h...,0
1,nhiệt_tình dễ hiểu đúng giờ,0
2,những gì được học nó nhằm mục_đích gì,4
3,xinh quá,0
4,giảng_viên rất vui_vẻ thân_thiện nhiệt_tình ...,0
...,...,...
28731,chất_lượng môn_học phù_hợp,0
28732,tao bực mày lắm rồi đấy,1
28733,per anh ơi đừng gym nữa boxing là đụ rùi,2
28734,thầy dạy vui lắm,0


# Tokenize

In [68]:
import torch
from torch.utils.data import TensorDataset
from transformers import AutoTokenizer
from underthesea import word_tokenize
from pyvi import ViTokenizer

def hybrid_tokenize(text):
    underthesea_tokens = word_tokenize(text, format="text").split() 
    pyvi_tokens = ViTokenizer.tokenize(text).split()
    
    merged_tokens = []
    
    i, j = 0, 0
    while i < len(underthesea_tokens) and j < len(pyvi_tokens):
        ut_token = underthesea_tokens[i]
        pv_token = pyvi_tokens[j]

        if ut_token == pv_token:
            merged_tokens.append(ut_token)
            i += 1
            j += 1
        else:
            if len(ut_token) >= len(pv_token):
                merged_tokens.append(ut_token)
                i += 1
                j += len(pv_token.split('_'))
            else:
                merged_tokens.append(pv_token)
                j += 1
                i += len(ut_token.split('_'))
    
    merged_tokens += underthesea_tokens[i+1:] + pyvi_tokens[j+1:]
    return merged_tokens

text = "Tôi cảm thấy cô đơn trong cuộc sống này"
merged_tokens = hybrid_tokenize(text)
print("Merged Tokens:", merged_tokens)


e:\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Merged Tokens: ['Tôi', 'cảm_thấy', 'cô_đơn', 'trong', 'cuộc_sống', 'này']


In [69]:
from sklearn.model_selection import train_test_split

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

def tokenize_data(df):
    tokenized_texts = []
    attention_masks = []
    labels = []
    
    for index, row in df.iterrows():
        text = row['Sentence'] 
        label = row['Label']  
        merged_tokens = hybrid_tokenize(text)
        encoding = tokenizer(' '.join(merged_tokens), truncation=True, padding="max_length", max_length=256, return_tensors="pt")
        tokenized_texts.append(encoding["input_ids"][0])
        attention_masks.append(encoding["attention_mask"][0])  
        labels.append(label)

    input_ids = torch.stack([torch.tensor(tokens) for tokens in tokenized_texts])
    attention_mask = torch.stack([torch.tensor(mask) for mask in attention_masks]) 
    labels = torch.tensor(labels)
    
    return TensorDataset(input_ids, attention_mask, labels)

train_df, test_df = train_test_split(data, test_size=0.2, stratify=data["Label"], random_state=42)
valid_df, test_df = train_test_split(test_df, test_size=0.5, stratify=test_df["Label"], random_state=42)

train_data = tokenize_data(train_df)
valid_data = tokenize_data(valid_df)
test_data = tokenize_data(test_df)

output_dir = os.path.join(BASE_DIR, "data/processed")
os.makedirs(output_dir, exist_ok=True)

torch.save(train_data, os.path.join(output_dir, "train.pt"))
torch.save(valid_data, os.path.join(output_dir, "valid.pt"))
torch.save(test_data, os.path.join(output_dir, "test.pt"))

C:\Users\TGDD\AppData\Local\Temp\ipykernel_17148\2482918352.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.stack([torch.tensor(tokens) for tokens in tokenized_texts])
C:\Users\TGDD\AppData\Local\Temp\ipykernel_17148\2482918352.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.stack([torch.tensor(mask) for mask in attention_masks])
